## 批量导出教师成绩分析表【糖葫芦图+箱图】★

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import os   

#%matplotlib inline
plt.rcdefaults()
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['figure.figsize']=[14,10] #改变显示图形的大小

In [2]:
file_name='data/xscjb20190215.csv'
fields=['XNM','XQM','JSGH','JSXM','KCH','KCMC','BFZCJ']
df_cjb_js_term=pd.read_csv(file_name,usecols=fields,dtype={'XNM':np.object,'XQM':np.object})

d:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_cjb_js_term.tail()
df_cjb_js_term.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3813515 entries, 0 to 3813514
Data columns (total 7 columns):
XNM      object
XQM      object
BFZCJ    float64
KCH      object
KCMC     object
JSGH     object
JSXM     object
dtypes: float64(1), object(6)
memory usage: 203.7+ MB


### 获取指定学期的数据（XNM='2018-2019'  XQM='3'）

In [30]:
#df_cjb_sinterm=df_cjb_js_term[(df_cjb_js_term.XNM=='2018') & (df_cjb_js_term.XQM=='3')]   #176923 rows × 7 columns
df_cjb_sinterm=df_cjb_js_term
df_cjb_sinterm.head()

,XNM,XQM,BFZCJ,KCH,KCMC,JSGH,JSXM
0,2003,12,79.0,10010002,体育（2）,NaN,无
1,2003,12,61.0,04120003,数学分析(Ⅱ),NaN,无
2,2003,12,46.0,04110003,程序设计基础,NaN,无
3,2003,12,86.0,10006,篮球技术与裁判（男子）,NaN,无
4,2003,12,65.0,03010002,大学英语（2）,NaN,无


#### 载入教师基本信息表

In [31]:
file_name_js='data/jsjbxxb20190214.csv'
df_teacher=pd.read_csv(file_name_js,usecols=['JGH','JG_ID'])
df_teacher.head()
df_teacher.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3132 entries, 0 to 3131
Data columns (total 2 columns):
JGH      3132 non-null object
JG_ID    2787 non-null object
dtypes: object(2)
memory usage: 49.0+ KB


#### 在成绩库中加入教师部门代码（只需执行1次）

In [32]:
df_cjb_sinterm=df_cjb_sinterm.join(df_teacher.set_index('JGH'),on='JSGH')
df_cjb_sinterm.head()

,XNM,XQM,BFZCJ,KCH,KCMC,JSGH,JSXM,JG_ID
0,2003,12,79.0,10010002,体育（2）,NaN,无,NaN
1,2003,12,61.0,04120003,数学分析(Ⅱ),NaN,无,NaN
2,2003,12,46.0,04110003,程序设计基础,NaN,无,NaN
3,2003,12,86.0,10006,篮球技术与裁判（男子）,NaN,无,NaN
4,2003,12,65.0,03010002,大学英语（2）,NaN,无,NaN


In [33]:
#获取部门代码
set_jgid=df_cjb_sinterm.JG_ID.drop_duplicates(keep='first')
set_jgid.sort_values(inplace=True)
set_jgid.values

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '12',
       '15', '16', '17', '21', '22', '23', '24', '25', '26', '27', '28',
       '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39',
       '45', '52', '98', '99', 'X5', 'X7', nan], dtype=object)

In [46]:
'''
list_bm=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '12',
       '15', '16', '17', '24', '28', '29', '30', '32', '33', '34', '36',
       '37', '38', '39', '52', '99']
'''
#部门代码 
list_bm=['99']

#list_err=[]
#list_err1=['201000023','200500047','201700030','200800025','201200028']   #2018-2019-1  出错的教师工号
list_err=['p00000074','p00000229','p00000016','910000404','p00000196','p00000040','p00000185',
          'p00000186','p00000187','200900046','201700006']
colors = ['yellow','pink', 'lightblue', 'lightgreen','lightyellow','goldenrod','darkolivegreen',
         'dodgerblue','darkcyan','lightgray','tomato','indianred','khaki','lime']
red_square = dict(markerfacecolor='r', marker='s')
#for jg_id in set_jgid:
for jg_id in list_bm:
    #建立部门代码目录
    path_name='img/thlt_xt/'+str(jg_id)
    isExists=os.path.exists(path_name)
    if not isExists:
        os.mkdir(path_name)

    ser_js_jgid=df_teacher[df_teacher.JG_ID==jg_id].JGH
    
    #糖葫芦图所用数据
    df_cjb_group=df_cjb_sinterm.groupby(['JSGH','KCMC','BFZCJ']).count()
    df_cjb_group.sort_values('KCMC')
    df_cjb_group.reset_index(inplace=True)

    for index in ser_js_jgid.index:
        jsgh=ser_js_jgid[index]
        print(jg_id,jsgh)
        
        if jsgh in list_err:
            pass
        else :
            df_cjb_single=df_cjb_group[df_cjb_group.JSGH==jsgh] 
            len1=len(df_cjb_single.groupby('KCMC').count())
            df_cjb_single.loc[0]=[jsgh,'A',0,1,1,1,1,1]     #人为插入一条无关记录，是为了箱图与糖葫芦图共享X轴，上下显示一致。
            df_cjb_single=df_cjb_single.sort_values('KCMC')
                
            if (len1 !=0):   #没有课程的教师不绘制图像
                fig=plt.figure(figsize=(4+len1,10))    #设置图像大小
                (ax2,ax1)=fig.subplots(2,1,sharex=True)
                #(ax2,ax1)=fig.subplots(2,1)
                fig.suptitle("单个教师全部课程成绩分析")
                
                #----------绘制箱图----------------------
                df_cjb_xt=df_cjb_sinterm[df_cjb_sinterm.JSGH==jsgh]
                list_labels=df_cjb_xt.groupby('KCMC').count().sort_index(0).index.tolist()   #获取课程名称
                #print(list_labels)
                #生成箱图数据
                list_cjb_show=[]
                for kcmc in list_labels:
                    array_cjb=df_cjb_xt[df_cjb_xt.KCMC==kcmc].BFZCJ.to_numpy()
                    array_cjb.shape=(-1,1)    
                    list_cjb_show.append(array_cjb)

                ax2.grid()
                if len1 ==0:
                    pass
                elif len1==1:
                    show_data=list_cjb_show[0]
                else :
                    show_data=list_cjb_show

                dic_cjb_single=ax2.boxplot(show_data,
                                           meanline=True,
                                           showmeans=True,#显示平均分线
                                           patch_artist=True,  # fill with color
                                           labels=list_labels,  # will be used to label x-ticks
                                           flierprops=red_square) 
                 # fill with colors
                for patch, color in zip(dic_cjb_single['boxes'], colors):
                    patch.set_facecolor(color)    

                ax2.plot([-1,len1+1],[60,60],'r-.')
                plt.yticks(range(0,101,10))
                
                #------------绘制糖葫芦图---------
                point_size=80     #图像点的大小
                df_cjb_single.insert(0,'SCALE',df_cjb_single.KCH*point_size)  #增加按比例显示人数的列

                ax1.grid()
                ax1.scatter('KCMC','BFZCJ','SCALE',data=df_cjb_single)
                ax1.plot([-1,len1+1],[60,60],'r-.')
              
                plt.xlabel('课程名称')
                plt.ylabel('课程成绩')
                plt.yticks(range(0,101,10))
                #plt.xticks(range(0,len1,1),list_labels,rotation=20)
                plt.xticks(rotation=20)

                pic_file_name='img/thlt_xt/'+jg_id+'/'+jsgh+'任课成绩分析.png'
                plt.savefig(pic_file_name)
                plt.close()     #否则打开的图形太多
                #plt.show()

print('ok')

99 200800044


d:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
d:\ProgramData\Anaconda3\lib\site-packages\matplotlib\pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


99 200800049
99 200700003
99 200700006
99 200700007
99 200700074
99 200700031
99 200600044
99 200700030
99 200500003
99 200500002


d:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4291: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)
d:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py:1246: RuntimeWarning: invalid value encountered in less_equal
  wiskhi = np.compress(x <= hival, x)
d:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py:1253: RuntimeWarning: invalid value encountered in greater_equal
  wisklo = np.compress(x >= loval, x)
d:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py:1261: RuntimeWarning: invalid value encountered in less
  np.compress(x < stats['whislo'], x),
d:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py:1262: RuntimeWarning: invalid value encountered in greater
  np.compress(x > stats['whishi'], x)


99 200800054
99 200800014
99 200800095
99 200900021
99 200900018
99 200900028
99 200900047
99 200800074
99 200800019
99 200700089
99 200800104
99 200800048
99 200800041
99 200800033
99 200800101
99 200200022
99 199900011
99 200200007
99 199300012
99 200400035
99 199900035
99 199700004
99 200800003
99 200700015
99 200600046
99 200900052
99 200900051
99 201000015
99 200800059
99 200800077
99 200600022
99 198900217
99 199800013
99 200200027
99 199000174
99 199000173
99 200000035
99 200000002
99 200000018
99 199000180
99 200900053
99 200900039
99 200800008
99 198900006
99 198900007
99 198900026
99 198900027
99 198900029
99 198900033
99 200700049
99 200700093
99 200800056
99 200900056
99 201000020
99 201000028
99 201000064
99 201000075
99 201000070
99 201000035
99 201000038
99 201000068
99 201000034
99 201000056
99 201000057
99 199000091
99 198900045
99 198900060
99 198900063
99 198900066
99 198900070
99 198900116
99 198900121
99 198900123
99 198900130
99 198900259
99 199000050
99 199500027

##### --end---